### USD/JPYレートのライブアクションプロット

以下は、Pythonの `yfinance` ライブラリを使用してUSD/JPY為替レートのライブアクションプロットを実装したものです。このコードは、金融APIから最新のデータを取得し、リアルタイムでプロットを更新します。


In [1]:
import dash
from dash import dcc, html
from dash.dependencies import Output, Input, State
import yfinance as yf
import plotly.graph_objs as go
import pandas as pd

# Constants
ticker = yf.Ticker("USDJPY=X")

app = dash.Dash(__name__)
app.title = "USD/JPY Live Chart"

app.layout = html.Div([

    html.Div([
        dcc.Checklist(
            id='sma-toggle',
            options=[{'label': '5-min SMA', 'value': 'SMA_5'},
                     {'label': '10-min SMA', 'value': 'SMA_10'}],
            value=['SMA_5'],  # SMA shown by default
        ),
    ], style={'marginBottom': '10px'}),

    dcc.Graph(id='live-graph'),
    dcc.Interval(
        id='interval-component',
        interval=1*1000,  # 1 minute in milliseconds
        n_intervals=0
    )
])

@app.callback(
    Output('live-graph', 'figure'),
    Input('interval-component', 'n_intervals'),
    State('sma-toggle', 'value')
)
def update_graph(n, sma_toggle):
    try:
        
        data = ticker.history(period="1d", interval="1m").tail(20)
        data['SMA_5'] = data['Close'].rolling(window=5).mean()
        data['SMA_10'] = data['Close'].rolling(window=10).mean()

        fig = go.Figure()

        latest_time = data.index[-1]
        latest_rate = data['Close'].iloc[-1]

        # Add the figures
        fig.add_trace(go.Scatter(x=data.index, y=data['Close'], mode='lines+markers', name='USD/JPY'))

        # Conditionally add SMA line
        if 'SMA_5' in sma_toggle:
            fig.add_trace(go.Scatter(
                x=data.index,
                y=data['SMA_5'],
                mode='lines',
                name='5-min SMA',
                line=dict(dash='dash')
            ))
        
        if 'SMA_10' in sma_toggle:
            fig.add_trace(go.Scatter(
                x=data.index,
                y=data['SMA_10'],
                mode='lines',
                name='10-min SMA',
                line=dict(dash='dot')
            ))

        # Change y axis
        fig.update_yaxes(title_text='Rate', range=[data['Close'].min() - 0.1, data['Close'].max() + 0.1])

        # Label all x ticks
        fig.update_xaxes(tickvals=data.index, ticktext=[t.strftime('%H:%M') for t in data.index], tickangle=315)
        
        # Label latest point
        fig.add_trace(go.Scatter(
            x=[latest_time],
            y=[latest_rate],
            mode='markers+text',
            name='Current Rate',
            text=[f"{latest_rate:.4f}"],
            textposition="top center",
            marker=dict(size=10, color='red')
        ))

        fig.update_layout(title='USD/JPY (Live) with SMA', xaxis_title='Time', yaxis_title='Rate')

        return fig

    except Exception as e:
        print("Error:", e)
        return go.Figure()

if __name__ == '__main__':
    app.run(debug=True)

In [2]:
import yfinance as yf
import ipywidgets as widgets
from IPython.display import display, clear_output

# The list to store values
my_list = []

# Output widget to show list updates
output = widgets.Output()

# Button widget
add_button = widgets.Button(description="Trade")

# Function to handle button click
def on_button_click(b):

    # Get the latest USD/JPY rate
    latest_close = (
        yf.Ticker("USDJPY=X")
        .history(period="1d", interval="1m")
        .tail(1)
        .iloc[-1]['Close']
    )
    my_list.append(f'{latest_close:.4f}')
    with output:
        clear_output()
        print(f"Current list: {my_list}")

# Assign the click event
add_button.on_click(on_button_click)

# Display the widgets
display(add_button, output)


Button(description='Trade', style=ButtonStyle())

Output()

In [ ]:
# Get profit

def calc_profit(trades):
    if len(trades) < 2:
        return "Not enough data to calculate profit."
    
    # Convert strings to floats
    trades = [float(x) for x in trades]

    buys = [trades[i] for i in range(0, len(trades), 2)]
    sells = [trades[i] for i in range(1, len(trades), 2)]
    
    # Vector of buy-sells
    profit = sum(sells[i] - buys[i] for i in range(len(buys)))

    return f"Profit: {profit:.4f}"

calc_profit(my_list)